In [1]:
import pandas as pd
from random import randint
from math import log
import json

In [2]:
Df = pd.read_csv("./databreach.csv")
Df["Date"] = pd.to_datetime(Df["Date"])
Df["data_classes"] = [ i.split(",") for i in Df["data_classes"] ]

In [3]:
Df.head()

,Date,Org,category,breach_count(Million),employee_count,data_classes
0,2017-10-01,Yahoo,web services,3000.0,8600.0,[Security questions and answers]
1,2018-03-01,UIDAI,public welfare,1100.0,NaN,"[Names, Aadhar number, Bank account numbers]"
2,2014-01-01,Yahoo,web services,500.0,8600.0,"[Names, Email addresses, Phone numbers, Passwo..."
3,2013-06-01,MySpace,social media,360.0,150.0,"[Names, Usernames, Dates of birth]"
4,2018-06-01,Exactis,web services,340.0,NaN,"[Phone numbers, Physical addresses, Email addr..."


### group by year

In [4]:
Df["year"] = Df.Date.dt.year
Df_year = pd.DataFrame(Df.groupby("year")["Date"].agg("count"))
Df = Df.drop(["year"], axis=1)


In [5]:
Df_year.head()

,Date
year,
2012,1
2013,3
2014,2
2016,3
2017,2


In [6]:
Df_year.to_csv("./final/year_bar_chart.csv")

### generate mock data for missing employee counts

In [7]:
temp = []
for i in Df["employee_count"]:
    try:
        temp.append(int(i))
    except:
        temp.append(randint(0,1000))
Df["employee_count"] = temp

### scatter plot and exposure index

In [8]:
log(3)

1.0986122886681098

In [9]:
Df['exposure_index'] = [
                        log((Df["breach_count(Million)"][num])*len(i))
                        for num, i in enumerate(Df["data_classes"])
                        ]
Df_scatter = Df[["exposure_index", "employee_count", "category", "Org"]]


In [10]:
Df_scatter.head()

,exposure_index,employee_count,category,Org
0,8.006368,8600,web services,Yahoo
1,8.101678,527,public welfare,UIDAI
2,8.006368,8600,web services,Yahoo
3,6.984716,150,social media,MySpace
4,7.620705,486,web services,Exactis


In [11]:
Df_scatter.to_csv("./final/scatter_data.csv")

### network graph


In [12]:
#  importing categories
Df_cats = pd.read_csv("./Classes_to_Cats.csv")
Df_cats["Data_category"] = [i.split(",") for i in Df_cats["Data_category"]]
Df_cats.head()

,Data_classes,Data_category
0,Email addresses,[PII]
1,Passwords,[passwords]
2,Usernames,[PII]
3,IP addresses,[device info]
4,Names,[PII]


In [13]:
# unique cats
unique_cats = {}
for i in Df_cats["Data_category"]:
    for j in i:
        if j not in unique_cats:
            unique_cats[j] = 0
unique_cats = list(unique_cats.keys())
unique_cats

['PII',
 'passwords',
 'device info',
 'age',
 'geolocation',
 'demographics',
 'consumer behavior',
 'personal communication',
 'employment history',
 'financial info',
 'health',
 'social',
 'demographic',
 'password']

In [14]:
class_to_cat = {}
for i in Df_cats.iterrows():
    class_to_cat[i[1][0]] = i[1][1][0]

In [15]:
# data_cats exposed
cats_exposed = []
for i in Df.data_classes:
    temp = []
    for j in i:
        if class_to_cat[j] not in temp:
            temp.append(class_to_cat[j])
    cats_exposed.append(temp)
Df["data_cats"] = cats_exposed
Df.head()

,Date,Org,category,breach_count(Million),employee_count,data_classes,exposure_index,data_cats
0,2017-10-01,Yahoo,web services,3000.0,8600,[Security questions and answers],8.006368,[passwords]
1,2018-03-01,UIDAI,public welfare,1100.0,527,"[Names, Aadhar number, Bank account numbers]",8.101678,"[PII, financial info]"
2,2014-01-01,Yahoo,web services,500.0,8600,"[Names, Email addresses, Phone numbers, Passwo...",8.006368,"[PII, passwords, age]"
3,2013-06-01,MySpace,social media,360.0,150,"[Names, Usernames, Dates of birth]",6.984716,"[PII, age]"
4,2018-06-01,Exactis,web services,340.0,486,"[Phone numbers, Physical addresses, Email addr...",7.620705,"[PII, geolocation, consumer behavior, demograp..."


In [16]:
final_network_data = []

In [17]:
for i in unique_cats:
    count = 0
    for j in unique_cats:
        if i != j:
            for point in Df.data_cats:
                if (i in point) and (j in point):
                    final_network_data.append({
                                    "data":{
                                            'source': i.lower(),
                                            'target': j.lower()
                                            }
                                })
                    count += 1
    if count:
        final_network_data.append({
                            'data':{
                                    'id': i.lower(),
                                    'label':i
                                    }
                            })

In [18]:
# saving json
json_object = json.dumps(final_network_data, indent = 4)
with open("./final/network_data.json", "w") as outfile:
    outfile.write(json_object)